In [5]:
%matplotlib inline
import pandas as pd
from astropy.table import Table
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
from astropy.io import fits
from specbox import SpecSDSS
import multiprocessing

In [2]:
def sdss_spname(plate, mjd, fiberid):
#     if plate <= 2975:
#         plate = '{:0>4}'.format(plate)
    filename = 'spec-{:0>4}-{}-{:0>4}.fits'.format(plate, mjd, fiberid)
    return filename

In [11]:
new_wave = np.linspace(2000, 3199, num=1200) * u.AA 

def spectransform(filename, z):
    try:
        sp = SpecSDSS(filename)
        sp.to_restframe(z, inplace=True)
        sp.trim([1950, 3250])
        sp.flux_conserve_resample(new_wave, True)
        idx = np.isnan(sp.flux.value)
        sp.flux.value[idx] = np.nanmedian(sp.flux.value)
        sp.smooth(5, 3, inplace=True, plot=False)
        f_norm = stats.zscore(sp.spec.flux.value)
    except:
        f_norm = np.zeros(1200)
    return f_norm

In [7]:
np.zeros(1200).shape

(1200,)

### Read `felostrong_all.csv` or something similar



In [3]:
flux_list = []
flist = basepath+df.relapath
flist = flist[:50]

In [1]:
# from os.path import basename
# for i, item in enumerate(flist):
#     f_norm = spectransform(item)
#     filename = basename(item)
#     flux_list.append(f_norm)
#     print("No. {}: file {} done.".format(i, filename))

In [16]:
flux_list[0]

array([ 1.2376996,  1.2675174,  1.2771946, ..., -1.5064276, -1.3667287,
       -1.4670243], dtype=float32)

In [7]:
df = pd.DataFrame(flux_list)

In [8]:
df['filename'] = flist

In [9]:
df

,filename


In [17]:
df1 = pd.read_csv('./mlFeLo/cnn/spec/train_v3/train_all_bin_p1.csv')

In [18]:
df1

,Filename,Label,redshift
0,spec-7378-56715-0436.fits,FeLoBAL,2.239379
1,spec-3763-55508-0990.fits,FeLoBAL,2.315665
2,spec-3771-55544-0842.fits,FeLoBAL,1.567929
3,spec-11677-58510-0991.fits,FeLoBAL,1.588018
4,spec-4014-55630-0880.fits,FeLoBAL,2.244618
...,...,...,...
834,spec-5962-56265-0056.fits,Non-FeLoBAL,2.657000
835,spec-3655-55240-0764.fits,Non-FeLoBAL,2.510000
836,spec-7107-56740-0512.fits,Non-FeLoBAL,4.015000
837,spec-5059-56190-0562.fits,Non-FeLoBAL,2.154000


In [19]:
fullpath = './mlFeLo/cnn/spec/train_v3/' + df1['Filename']

In [20]:
fullpath

0      ./mlFeLo/cnn/spec/train_v3/spec-7378-56715-043...
1      ./mlFeLo/cnn/spec/train_v3/spec-3763-55508-099...
2      ./mlFeLo/cnn/spec/train_v3/spec-3771-55544-084...
3      ./mlFeLo/cnn/spec/train_v3/spec-11677-58510-09...
4      ./mlFeLo/cnn/spec/train_v3/spec-4014-55630-088...
                             ...                        
834    ./mlFeLo/cnn/spec/train_v3/spec-5962-56265-005...
835    ./mlFeLo/cnn/spec/train_v3/spec-3655-55240-076...
836    ./mlFeLo/cnn/spec/train_v3/spec-7107-56740-051...
837    ./mlFeLo/cnn/spec/train_v3/spec-5059-56190-056...
838    ./mlFeLo/cnn/spec/train_v3/spec-6403-56357-048...
Name: Filename, Length: 839, dtype: object

In [21]:
pool = multiprocessing.Pool(processes=80)
outputs = pool.map(spectransform, fullpath.values)

/gpfs/share/home/1900011612/anaconda3/lib/python3.9/site-packages/specbox/basemodule/basemodule.py:308: RuntimeWarning: divide by zero encountered in power
  self.err = data['ivar']**-0.5 * 1e-17
/gpfs/share/home/1900011612/anaconda3/lib/python3.9/site-packages/specbox/basemodule/basemodule.py:308: RuntimeWarning: divide by zero encountered in power
  self.err = data['ivar']**-0.5 * 1e-17
/gpfs/share/home/1900011612/anaconda3/lib/python3.9/site-packages/specbox/basemodule/basemodule.py:308: RuntimeWarning: divide by zero encountered in power
  self.err = data['ivar']**-0.5 * 1e-17
/gpfs/share/home/1900011612/anaconda3/lib/python3.9/site-packages/specbox/basemodule/basemodule.py:308: RuntimeWarning: divide by zero encountered in power
  self.err = data['ivar']**-0.5 * 1e-17
/gpfs/share/home/1900011612/anaconda3/lib/python3.9/site-packages/specbox/basemodule/basemodule.py:308: RuntimeWarning: divide by zero encountered in power
  self.err = data['ivar']**-0.5 * 1e-17
/gpfs/share/home/190

In [22]:
dfspec = pd.DataFrame(outputs)

In [23]:
dfspec.shape

(839, 3522)

In [24]:
df1.columns

Index(['Filename', 'Label', 'redshift'], dtype='object')

In [25]:
# dfs = df.loc[:,['RA', 'DEC', 'PLATE', 'MJD', 'FIBERID', 'Z']]

In [26]:
dfspec = dfspec.join(df1)

In [27]:
dfspec.head()

,0,1,2,3,4,5,6,7,8,9,...,3515,3516,3517,3518,3519,3520,3521,Filename,Label,redshift
0,-0.272950,1.056850,0.541104,-0.425962,-0.290942,0.247559,0.736341,0.315651,0.096573,-0.118562,...,2.396678,2.723467,2.328005,2.140936,2.422459,2.835612,2.595600,spec-7378-56715-0436.fits,FeLoBAL,2.239379
1,0.437413,0.597947,0.765858,0.817063,0.831437,0.991502,1.048408,1.143962,1.006748,0.793228,...,-0.813932,-0.900805,-1.067298,-1.159893,-1.012454,-0.836310,-0.565390,spec-3763-55508-0990.fits,FeLoBAL,2.315665
2,-1.432960,-1.024800,-1.287398,-1.660437,-1.546280,-1.313383,-1.302834,-1.382930,-1.353146,-1.317502,...,1.380772,1.382211,1.287288,1.102472,1.093720,1.348487,1.024443,spec-3771-55544-0842.fits,FeLoBAL,1.567929
3,-4.122627,-3.528450,-3.133138,-2.783693,-2.433680,-1.853307,-1.583229,-1.570417,-1.673850,-1.753690,...,-0.426877,-0.280805,-0.126278,-0.211044,-0.273270,-0.100862,0.005425,spec-11677-58510-0991.fits,FeLoBAL,1.588018
4,-0.589485,-0.450383,-0.775628,-1.284829,-1.646684,-1.498073,-1.014081,-0.498082,-0.015552,0.352968,...,0.178782,0.253349,0.477122,0.291706,0.143951,0.471743,0.852256,spec-4014-55630-0880.fits,FeLoBAL,2.244618


In [28]:
dfspec.dropna(subset=pd.Index(np.arange(3522)), inplace=True)

In [29]:
dfspec.reset_index(drop=True, inplace=True)

In [30]:
# dfspec.to_csv('train_v3_fnorm.csv', index=False)

In [31]:
dfspec.to_csv('./Datasets/train_v3_fnorm.csv', index=False)

In [32]:
dftest = pd.read_csv('./Datasets/train_v3_fnorm.csv')

In [33]:
# dftest

In [34]:
arr1 = dfspec.loc[:,pd.Index(np.arange(3522))].values

In [35]:
arr1.mean(axis=0)

array([ 0.62664549,  0.58049722,  0.54389899, ..., -0.54370868,
       -0.37341465, -0.32837646])

In [36]:
dftest.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3515', '3516', '3517', '3518', '3519', '3520', '3521', 'Filename',
       'Label', 'redshift'],
      dtype='object', length=3525)

In [46]:
dftest.values[:,0:3522].astype(np.float)

/tmp/ipykernel_76470/424564734.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dftest.values[:,0:3522].astype(np.float)


array([[-0.27295032,  1.0568504 ,  0.54110444, ...,  2.42245889,
         2.8356123 ,  2.59560013],
       [ 0.4374127 ,  0.5979466 ,  0.76585793, ..., -1.01245391,
        -0.83630961, -0.56538969],
       [-1.4329598 , -1.0248    , -1.287398  , ...,  1.09371972,
         1.34848714,  1.02444327],
       ...,
       [-0.9309967 , -1.1451778 , -1.2735512 , ..., -0.21071447,
        -0.30297497,  0.02675193],
       [-1.2140468 ,  0.91633946,  0.26268247, ...,  0.54616404,
         1.06772721,  0.18743181],
       [-1.4852076 , -2.4699316 , -2.462683  , ..., -0.18074313,
         0.1722443 , -0.13230458]])